In [0]:
## Gostaria de deixar alguns adendos , sobre a tratativa do desafio técnico:
## Toda estrutura foi criada na Nuvem pela ferramenta Google Colab , a extensão de arquivos é .IPYNB
## Onde alguns imports são da própria ferramenta. Não iram executar diretamente no Python.
## O google solicita o authorization code , para montar a partição do Google Drive , para acesso aos arquivos
## O comando spark-subit não irá se aplicar neste caso, para execução do código.
## Qualquer dúvida , fico a disposição.
## Att. Bruno Milhati Cavallini





In [0]:
# Download do Pacote .Tgz do Spark
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
#Instalação Java-8 , instalação Spark , pacotes findsaprk , collections
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install collections-extended

In [0]:
## Setando variaveis de ambiente 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
# imports
import findspark
findspark.init()
import pandas as pd
import collections
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import col
from pyspark.sql.functions import sum as spark_sum
from pyspark.sql.functions import unix_timestamp, from_unixtime

# SparkContext
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()


In [0]:
# Leitura de arquivos do Google Drive
# Ao executar o codigo o google solicita o authorization code , para montar a partição do Google Drive 
# Exemplo: 4/tQFvkBi4HJjeLRJMCwMOAkXIGBeMeqmpA15ir6t2i-trbmeMLYZ-lxc 

from google.colab import drive 
from google.colab import files
drive.mount('/content/gdrive')

file_loc1 = 'gdrive/My Drive/NASA_access_log_Jul95.gz'
file_loc2 = 'gdrive/My Drive/NASA_access_log_Aug95.gz'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Rdd Texto
linesj = sc.textFile(file_loc1)
linesa = sc.textFile(file_loc2)


In [0]:
## Número de hosts únicos.
Julho_count = linesj.flatMap(lambda line: line.split(' ')[0]).distinct().count()
agosto_count = linesa.flatMap(lambda line: line.split(' ')[0]).distinct().count()
print('Numero de Hosts unicos em julho: %s' % Julho_count)
print('Numero de Hosts unicos em agosto: %s' % agosto_count)

Numero de Hosts unicos em julho: 55
Numero de Hosts unicos em agosto: 53


In [0]:
##O total de erros 404.
j404j = linesj.filter(lambda x: 'HTTP/1.0" 404' in x).count()
j404a = linesa.filter(lambda x: 'HTTP/1.0" 404' in x).count()
print('O total de erros 404 ocorridos em julho: %s' % j404j)
print('O total de erros 404 ocorridos em agosto: %s' % j404a)


O total de erros 404 ocorridos em julho: 10754
O total de erros 404 ocorridos em agosto: 9944


In [0]:
# DataFrame
df1 = spark.read.text(file_loc1)
df2 = spark.read.text(file_loc2)

In [0]:
## Expressão Regular
re_host = r'(^\S+\.[\S+\.]+\S+)\s' #Host
re_timestamp = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]' # Timestamp
re_url = r'\"(\S+)\s(\S+)\s*(\S*)\"' #url -Requisição 
re_codigo = r'\s(\d{3})\s' # codigo http
re_bytes = r'\s(\d+)$' # bytes

In [0]:
sql = df1.select(regexp_extract('value', re_host, 1).alias('host'),
                         regexp_extract('value', re_timestamp, 1).alias('timestamp'),
                         regexp_extract('value', re_url, 2).alias('url'),
                         regexp_extract('value', re_codigo, 1).cast('integer').alias('codigo'),
                         regexp_extract('value', re_bytes, 1).cast('integer').alias('bytes'))

In [0]:
sql2 = df2.select(regexp_extract('value', re_host, 1).alias('host'),
                         regexp_extract('value', re_timestamp, 1).alias('timestamp'),
                         regexp_extract('value', re_url, 2).alias('url'),
                         regexp_extract('value', re_codigo, 1).cast('integer').alias('codigo'),
                         regexp_extract('value', re_bytes, 1).cast('integer').alias('bytes'))

In [0]:
df1.show(20, truncate=False)

In [0]:
sql.show()

In [0]:
##1. Número de hosts únicos. Julho e Agosto
##2. O total de erros 404. Julho e Agosto
##3. Os 5 URLs que mais causaram erro 404. Julho e Agosto
##4. Quantidade de erros 404 por dia. Julho e Agosto
##5. O total de bytes retornados.Julho e Agosto 

In [0]:
sql.groupBy('host').count().filter('count = 1').select('host').show()
sql2.groupBy('host').count().filter('count = 1').select('host').show()

In [0]:
sql.groupBy('codigo').count().filter('codigo = "404"').show()
sql2.groupBy('codigo').count().filter('codigo = "404"').show()

In [0]:
sql.filter('codigo = "404"').groupBy('url').count().sort(col("count").desc()).show(5, truncate=False)
sql2.filter('codigo = "404"').groupBy('url').count().sort(col("count").desc()).show(5, truncate=False)

In [0]:
sql.filter('codigo = "404"').groupBy(sql.timestamp.substr(1,11).alias('day')).count().show()
sql2.filter('codigo = "404"').groupBy(sql2.timestamp.substr(1,11).alias('day')).count().show()

In [0]:
sql.select('bytes').groupBy().sum().show()
sql2.select('bytes').groupBy().sum().show()

In [0]:
! git clone https://github.com/souvik3333/Testing-and-Debugging-Tools 


Cloning into 'Testing-and-Debugging-Tools'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 247 (delta 24), reused 8 (delta 2), pack-reused 163
Receiving objects: 100% (247/247), 28.47 MiB | 26.12 MiB/s, done.
Resolving deltas: 100% (48/48), done.
